# Interview Part 1 : SQL Questions

In [2]:
from google.colab import auth
auth.authenticate_user()
print('Authenticated')

Authenticated


Note: I added the dataset to Google BigQuery and connected the data using Google Colab notebook. 
`carbide-algebra-274912.interview.adops` is the name of the table on bigquery

**Question 1:** Consider only the rows with country_id = "BDV" (there are 844 such rows). For each site_id, we can compute the number of unique user_id's found in these 844 rows. Which site_id has the largest number of unique users? And what's the number?

In [9]:
from google.cloud import bigquery
project_id='carbide-algebra-274912'
client=bigquery.Client(project=project_id)
dfq1=client.query('''SELECT site_id,(count(distinct(user_id))) as NumberOfUniqueUsers FROM `carbide-algebra-274912.interview.adops` where country_id='BDV' group by site_id order by NumberOfUniqueUsers desc limit 1''').to_dataframe()
print(dfq1)

  site_id  NumberOfUniqueUsers
0   5NPAU                  544


**Question 2:** Between 2019-02-03 00:00:00 and 2019-02-04 23:59:59, there are four users who visited a certain site more than 10 times. Find these four users & which sites they (each) visited more than 10 times. (Simply provides four triples in the form (user_id, site_id, number of visits) in the box below.)

In [26]:
dfq2=client.query('''select user_id,site_id,count(ts) as NumberOfTimesVisited from `carbide-algebra-274912.interview.adops` where ts>'2019-02-03 00:00:00' and ts<'2019-02-04 23:59:59' group by user_id,site_id having count(ts)>10 order by NumberOfTimesVisited desc''').to_dataframe()
print(dfq2)

  user_id site_id  NumberOfTimesVisited
0  LC3A59   N0OTG                    26
1  LC06C3   N0OTG                    25
2  LC3C9D   N0OTG                    17
3  LC3C7E   3POLC                    15


Observation: On the given day three of the most visited users visited the same site N0OTG. I ran an additional query to see the site with most number of unique visitors and found it to be 5NPAU followed by N0OTG. I would research further to see why users LC3A59, LC06C3 and LC3C9D visited N0OTG these many times on this given day.

**Question 3**: For each site, compute the unique number of users whose last visit (found in the original data set) was to that site. For instance, user "LC3561"'s last visit is to "N0OTG" based on timestamp data. Based on this measure, what are top three sites? (hint: site "3POLC" is ranked at 5th with 28 users whose last visit in the data set was to 3POLC; simply provide three pairs in the form (site_id, number of users).)

In [17]:
dfq3=client.query('''SELECT site_id, COUNT( distinct user_id) as user_count FROM `carbide-algebra-274912.interview.adops` AS v
                  where ts=(SELECT MAX(ts) 
                    FROM `carbide-algebra-274912.interview.adops` as w where v.user_id=w.user_id
                ) 
                GROUP BY v.site_id
                ORDER BY user_count DESC
                LIMIT 3''').to_dataframe()
print(dfq3)                

  site_id  user_count
0   5NPAU         992
1   N0OTG         561
2   QGO3G         289


**Question 4:** For each user, determine the first site he/she visited and the last site he/she visited based on the timestamp data. Compute the number of users whose first/last visits are to the same website. What is the number?

In [24]:
dfq4=client.query('''select count(*) as NumerOfUsers from `carbide-algebra-274912.interview.adops` a inner join `carbide-algebra-274912.interview.adops` b on a.user_id=b.user_id where
                    a.ts= (SELECT MAX(ts) 
                    FROM `carbide-algebra-274912.interview.adops` as c where a.user_id=c.user_id) 
                    and b.ts= (SELECT MIN(ts) 
                    FROM `carbide-algebra-274912.interview.adops` as d where b.user_id=d.user_id) 
                    and a.site_id=b.site_id and a.user_id=b.user_id ''').to_dataframe()
print(dfq4)                    

   NumerOfUsers
0          1670
